In [4]:
import pandas as pd
import requests
import bs4

from selenium import webdriver

In [5]:
class Currency_Parsing:
    
    BASIC_URL = 'https://www.priorbank.by/offers/services/currency-exchange'    
    DRIVER = webdriver.Chrome("/home/felipe/Downloads/chromedriver")
    SAVE_LINK = r"../priorbank_currency_exchange.csv"

    def __init__(self, url: str = None, driver: webdriver = None, save_link: str = None):
        self.url = url if url != None else self.BASIC_URL
        self.driver = driver if driver != None else self.DRIVER
        self.save_link = save_link if save_link != None else self.SAVE_LINK
        self.df: pd.DataFrame
        
    def save_dataframe_csv(self, df: pd.DataFrame):
        df.to_csv(self.save_link, sep="\t", index=False)
        return self

    def parse_rows(self, exchange_way: str, div_rows: bs4.element.ResultSet, is_conversion: bool) -> pd.DataFrame:
        df = pd.DataFrame(columns=['exchange_way', 'currency', 'buy', 'sell', 'buy_sell', 'conversion'])
        conversion_value = 0
        if is_conversion:
            conversion_value = 1
            buy_sell_value = 0
        else: 
            conversion_value = 0
            buy_sell_value = 1

        columns = div_rows.find_all('div', attrs={'class': "homeModuleColumn"})
        currency_div_values = columns[0].find_all('p')
        buy_price_div = columns[1].find_all('p')
        sell_price_div = columns[2].find_all('p')

        for j in range(1, len(currency_div_values)):
            currency = currency_div_values[j].getText()
            sell_price = sell_price_div[j].getText()
            buy_price = buy_price_div[j].getText()
            df.loc[len(df)] = [exchange_way, currency, buy_price, sell_price, buy_sell_value, conversion_value]
        return df

    def create_currency_dataframe(self) -> pd.DataFrame:
        df = pd.DataFrame(columns=['exchange_way', 'currency', 'buy', 'sell', 'buy_sell', 'conversion'])
        # открываю через selenium, тк через bs4 у меня почему-то не отображаются цифры 
        self.driver.get(self.url)
        driver_parser = bs4.BeautifulSoup(self.driver.page_source)

        exchange_way_categories = driver_parser.find('ul', attrs={'class': 'toggle__list'})
        
        list_exchange_way = exchange_way_categories.find_all('li')
        divs_exchange_way = driver_parser.find_all('div', attrs={'class': 'smartfox--calc'})

        # [цифровой банк, по карточке, наличные]
        for i in range(len(list_exchange_way)):
            way = list_exchange_way[i].getText()
            # у раздела наличные специфический интерфейс
            if i == len(list_exchange_way) - 1:
                values_currency_rows = divs_exchange_way[i].find_all('div', attrs={'class': 'homeModuleRow--curr'})
                for values_currency_row in values_currency_rows:
                    df = pd.concat([df, self.parse_rows(exchange_way=way, div_rows=values_currency_row, is_conversion = False)])
                conversion_row = divs_exchange_way[i].find_all('div', attrs={'class': "homeModuleRow"})[3]
                df = pd.concat([df, self.parse_rows(exchange_way=way, div_rows=conversion_row, is_conversion = True)])
            else:
                buy_sell_row = divs_exchange_way[i].find_all('div', attrs={'class': "homeModuleRow"})[0]
                df = pd.concat([df, self.parse_rows(exchange_way=way, div_rows=buy_sell_row, is_conversion = False)])
                conversion_row = divs_exchange_way[i].find_all('div', attrs={'class': "homeModuleRow"})[1]
                df = pd.concat([df, self.parse_rows(exchange_way=way, div_rows=conversion_row, is_conversion = True)])
    

        self.df = df.reset_index(drop=True)
        self.save_dataframe_csv(self.df)
        return self.df



/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_92211/130383257.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  DRIVER = webdriver.Chrome("/home/felipe/Downloads/chromedriver")


In [6]:
currency_parsing = Currency_Parsing()
currency_parsing.create_currency_dataframe()

,exchange_way,currency,buy,sell,buy_sell,conversion
0,Цифровой банк,USD,2.907,2.941,1,0
1,Цифровой банк,EUR,3.132,3.18,1,0
2,Цифровой банк,RUB за 100 RUB,2.7,3.76,1,0
3,Цифровой банк,USD / RUB,75.76,107.4,0,1
4,Цифровой банк,EUR / USD,1.065,1.095,0,1
5,Цифровой банк,EUR / RUB,81.95,116.78,0,1
6,По карточке,USD,2.907,2.941,1,0
7,По карточке,EUR,3.132,3.18,1,0
8,По карточке,RUB за 100 RUB,2.7,3.76,1,0
9,По карточке,USD / RUB,75.76,107.4,0,1
